In [ ]:
from html_table_parser import parser_functions
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
dataframe = pd.DataFrame()
import collections
collections.Callable = collections.abc.Callable
from bs4 import BeautifulSoup
from urllib.request import urlopen

#저장할 데이터 프레임 제작
dataframe = pd.DataFrame()

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


Id = []#아이디 입력
Pw = []#비밀번호 입력



### Step 1. 로그인 페이지 접속
driver.get("https://busandbtceo.itgou.co.kr/member/login")
Id_count = len(Id)
for i in range(Id_count):
    # 아이디와 비밀번호 입력
    Id_num = Id[i]
    Pw_num = Pw[i]
    driver.find_element(By.ID, "login").send_keys(Id_num)
    driver.find_element(By.ID, "pw").send_keys(Pw_num)
    # 로그인 버튼 클릭 (웹사이트에 맞게 수정 필요)
    driver.find_element(By.ID, "btn_login").click()

    ### Step 2. 매장관리 버튼 클릭 (탭 안에서 매장관리 버튼 id 찾기)
    driver.find_element(By.XPATH, '//*[@id="tab-1"]/div[1]/div[1]/button[1]/a').click()

    ### Step 3. 주문통계 - 검색 조건 설정
    driver.switch_to.window(driver.window_handles[-1])
    driver.find_element(By.XPATH, '//*[@id="sh_nav_menu"]/div/ul/li[6]/ul/li[1]/a').click()
    #년도 옵션 선택
    option_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[2]/div[8]/div[1]/div[2]/select/option")
    option_class_count = len(option_class_elements)
    for o in range(2,option_class_count+1,1):
        option_path = f'/html/body/div[1]/div[2]/div[8]/div[1]/div[2]/select/option[{o}]'
        driver.find_element(By.XPATH, option_path).click()
        #상반기, 하반기 선택
        for l in range(1,3,1):
            season_path =  f'/html/body/div[1]/div[2]/div[8]/div[1]/div[2]/div/button[{l}]'
            driver.find_element(By.XPATH, season_path).click()
            # 배달완료 버튼 클릭
            driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[8]/div[1]/div[1]/select[2]/option[3]').click()

            # 검색 버튼 클릭
            driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[8]/div[1]/div[3]/button').click()
            time.sleep(1)
            while True:
                try:
                    # 데이터 크롤링 시작
                    tr_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[@class]")
                    tr_class_count = len(tr_class_elements)
                    for i in range(2,tr_class_count*2+1,2):
                        xpath = f'/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[{i}]'
                        driver.find_element(By.XPATH, xpath).click()
                        table = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div')
                        page = driver.page_source
                        
                        index = []
                        table1 = []

                        #주문번호, 배달구문 가져오기
                        Order_number_index = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[1]/td[1]')
                        Delivery_classification_index = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[1]/td[5]')
                        index.append(Order_number_index.text)
                        index.append(Delivery_classification_index.text)

                        #주문번호 배달구문 값 가져오기
                        Order_number_path = f'/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[{i}]/td[1]'
                        Delivery_classification_path = f'/html/body/div[1]/div[2]/div[8]/div[2]/div/table/tbody/tr[{i}]/td[5]'
                        Order_number = driver.find_element(By.XPATH, Order_number_path)
                        Delivery_classification = driver.find_element(By.XPATH, Delivery_classification_path)
                        table1.append(Order_number.text)
                        table1.append(Delivery_classification.text)
                        
                        #class st_basic_info의 클래스 가져오기
                        page = driver.page_source
                        soup = BeautifulSoup(page, "html.parser")
                        temp = soup.find_all('dt')
                        for element in temp:
                            index.append(element.text)
                            
                        #class st_basic_info의 데이터 가져오기
                        temp = soup.find_all('dd')
                        for element in temp:
                            table1.append(element.text)
                        
                        #1차 데이터 프레임 제작    
                        data = []
                        data.append(index)
                        data.append(table1)
                        dataframe1 = pd.DataFrame(data)
                        
                        #표 데이터 가져오기
                        temp = soup.find_all('div', 'menu_detail_tag')
                        p=parser_functions.make2d(temp[0])
                        
                        #메뉴 금액 수량 총금액만 남기기
                        df = pd.DataFrame(p)
                        df = df.dropna(axis=0)
                        df = df.drop([0])
                        df_reset = df.reset_index(drop=True)
                        
                        #팝업창 인덱스 가져오기
                        index1 = []
                        for j in range(1,5):
                            path =  f'/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr[1]/td[{j}]'
                            table_index = driver.find_element(By.XPATH, path)
                            index1.append(table_index.text)
                            
                        page = driver.page_source
                        soup = BeautifulSoup(page, "html.parser")
                        temp = soup.find_all('td', colspan='3')
                        for element in temp:
                            index1.append(element.text)  
                        a = len(index1)
                        df2 =  pd.DataFrame(index1)
                        df2 = df2.transpose()
                        
                        
                        #팝업창 표 안 데이터 가져오기
                        
                        if a == 8:#쿠폰 사용시
                            data2 = []
                            tr_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr")
                            tr_number = len(tr_class_elements)
                            for k in range(tr_number-3,tr_number+1):
                                path =  f'/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr[{k}]/td[2]'
                                table_index = driver.find_element(By.XPATH, path)
                                data2.append(table_index.text)
                            df3 = pd.DataFrame(data2)
                            df3 = df3.transpose()
                            
                        elif a == 9:#쿠폰 2개 사용시
                            data2 = []
                            tr_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr")
                            tr_number = len(tr_class_elements)
                            for k in range(tr_number-4,tr_number+1):
                                path =  f'/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr[{k}]/td[2]'
                                table_index = driver.find_element(By.XPATH, path)
                                data2.append(table_index.text)
                            df3 = pd.DataFrame(data2)
                            df3 = df3.transpose()
                            
                        elif a == 10:#쿠폰 2개 사용시
                            data2 = []
                            tr_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr")
                            tr_number = len(tr_class_elements)
                            for k in range(tr_number-5,tr_number+1):
                                path =  f'/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr[{k}]/td[2]'
                                table_index = driver.find_element(By.XPATH, path)
                                data2.append(table_index.text)
                            df3 = pd.DataFrame(data2)
                            df3 = df3.transpose()
                            
                        else:#쿠폰 미사용
                            data2 = []
                            tr_class_elements = driver.find_elements(By.XPATH, "/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr")
                            tr_number = len(tr_class_elements)
                            for k in range(tr_number-1,tr_number+1):
                                path =  f'/html/body/div[1]/div[3]/div[6]/div[1]/div[2]/div/div/table/tbody/tr[{k}]/td[2]'
                                table_index = driver.find_element(By.XPATH, path)
                                data2.append(table_index.text)    
                            df3 = pd.DataFrame(data2)
                            df3 = df3.transpose()
                        dataframe3 = pd.concat([df_reset, df3], axis = 1)
                        
                        #팝업창 인덱스와 데이터 합치기
                        dataframe3.columns = df2.columns
                        dataframe2 = pd.concat([df2, dataframe3], ignore_index=True)
                        empty_column = pd.Series(dtype=object)
                        if a == 8:
                            dataframe2.insert(7,'empty_column', empty_column)
                            dataframe2.insert(7,'empty_column2', empty_column)
                            
                        if a == 9:
                            dataframe2.insert(8,'empty_column', empty_column)
                                
                            
                        elif a <8:
                            dataframe2.insert(5,'empty_column', empty_column)
                            dataframe2.insert(5,'empty_column2', empty_column)
                            dataframe2.insert(5,'empty_column3', empty_column)
                            dataframe2.insert(5,'empty_column4', empty_column)
                            
                        result_df = pd.concat([dataframe1, dataframe2], axis = 1, ignore_index=True)
                        result_df = result_df.fillna(method='ffill')
                        result_df = result_df.drop(0)
                        
                        
                        dataframe = pd.concat([dataframe, result_df], axis = 0)
                        driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[6]/div[1]/div[1]/button').click()
                    driver.find_element(By.CLASS_NAME, 'paging_next').click()   
                except:
                    break
                
    #로그아웃
    driver.find_element(By.XPATH, '/html/body/nav/div/div[2]/a').click()
    
dataframe.columns = ['주문번호','배달구분','운영시간','결제방법','주소','가게 요청사항','배달 요청사항','주문시간','접수시간','배달시간','메뉴','금액','수량','총금액','배달팁','쿠폰총액','','','','결제금액']        
print(dataframe)
dataframe.to_csv("dataframe.csv", index = False, encoding='utf-8-sig')  
    
    